### Projet

In [2]:
pip install SPARQLWrapper

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
from SPARQLWrapper import SPARQLWrapper, JSON

def getSPARQLQuery():
    sparql_endpoint = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(sparql_endpoint)
    sparql.setQuery("""
    SELECT ?item ?itemLabel ?image {
    ?item wdt:P31 wd:Q144;  
            wdt:P18 ?image.  
    SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
    }""")

    sparql.setReturnFormat(JSON)
    res = sparql.query().convert()

    dog_array = []
    for result in res["results"]["bindings"]:
        dog_array.append(tuple((
            result["itemLabel"]["value"],
            result["image"]["value"])
        ))
    return dog_array

array = getSPARQLQuery()

{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q20709260'}, 'image': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Sallie%20Ann%20Jarrett%2C%20Civil%20War%20mascot.jpg'}, 'itemLabel': {'xml:lang': 'fr', 'type': 'literal', 'value': 'Sallie Ann Jarrett'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q20818039'}, 'image': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Tuna%20Melts%20My%20Heart.jpg'}, 'itemLabel': {'type': 'literal', 'value': 'Q20818039'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q21002371'}, 'image': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Doug%20the%20Pug%20NYC.jpg'}, 'itemLabel': {'type': 'literal', 'value': 'Q21002371'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q21978595'}, 'image': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Teddy%2C%20dog%20in%20sile

In [30]:
import pandas as pd

dataframe = pd.DataFrame(array, columns=["dog_name", "image"])
dataframe = dataframe.astype(
    dtype={"dog_name": "<U200", "image" : "<U200"}
)
dataframe["image"] = dataframe["image"].str.ljust(100)
print(dataframe)

               dog_name                                              image
0    Sallie Ann Jarrett  http://commons.wikimedia.org/wiki/Special:File...
1             Q20818039  http://commons.wikimedia.org/wiki/Special:File...
2             Q21002371  http://commons.wikimedia.org/wiki/Special:File...
3             Q21978595  http://commons.wikimedia.org/wiki/Special:File...
4                Kabosu  http://commons.wikimedia.org/wiki/Special:File...
..                  ...                                                ...
149                Bobi  http://commons.wikimedia.org/wiki/Special:File...
150     Primley Sceptre  http://commons.wikimedia.org/wiki/Special:File...
151          Q121549125  http://commons.wikimedia.org/wiki/Special:File...
152          Q123588410  http://commons.wikimedia.org/wiki/Special:File...
153                 Pep  http://commons.wikimedia.org/wiki/Special:File...

[154 rows x 2 columns]


In [35]:
import requests
import os


def download_image(image_url, directory):
    response = requests.get(image_url)
    if response.status_code == 200:
        image_filename = os.path.join(output_directory, os.path.basename(image_url))
        with open(image_filename, 'wb') as f:
            f.write(response.content)
        print(f"Image téléchargée avec succès : {image_filename}")
    else:
        print(f"Échec du téléchargement. Code d'état HTTP : {response.status_code}")


for index, row in dataframe.iterrows():
    image_url = row['image'][0]['value']
    download_image(image_url, 'images')

TypeError: string indices must be integers, not 'str'